# Задание 2

In [93]:
import pandas as pd
import numpy as np

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [100]:
# загрузим данные
df_gas = pd.read_csv(
    "/home/sk27/repo/eremeev/test_projects/new.csv", index_col=0
).reset_index(drop=True)

In [102]:
df_gas.head(3)

,date,receipt,product,group_of_products,store,city,sold_quantity,tunover_VAT,purchase_price_VAT,purchase_cost_VAT,margin_VAT,profit
0,2018-05-01 00:00:00,Чек ККМ ГС00-029452 от 01.05.2018 5:43:42,"Энерг.нап. Ягуар Фри 0,5 л ж/б",ЭНЕРГЕТИЧЕСКИЕ НАПИТКИ,АЗС № 1,Симферополь г,2,110.0,38.02,76.04,33.96,16.98
1,2018-05-01 00:00:00,Чек ККМ ГС00-029469 от 01.05.2018 8:37:13,Средство для разжигания огня Simex 0.25 л,Товары для отдыха,АЗС № 1,Симферополь г,1,60.0,24.00,24.00,36.00,36.00
2,2018-05-01 00:00:00,Чек ККМ ГС00-029495 от 01.05.2018 9:45:43,Средство для разжигания огня Simex 0.25 л,Товары для отдыха,АЗС № 1,Симферополь г,1,60.0,24.00,24.00,36.00,36.00


### Вариант 1 с циклом

In [103]:
%%time

df_gas["date"] = df_gas["date"].astype("datetime64[ns]")
index_list = []
for index, row in df_gas.iterrows():
    if row["tunover_VAT"] < 0:
        index_list.append(index)
        id_ = index - 1
        while True:
            if (
                df_gas.loc[id_, "city"] == df_gas.loc[index, "city"]
                and df_gas.loc[id_, "product"] == df_gas.loc[index, "product"]
                and df_gas.loc[id_, "group_of_products"]
                == df_gas.loc[index, "group_of_products"]
                and df_gas.loc[id_, "store"] == df_gas.loc[index, "store"]
                and df_gas.loc[id_, "tunover_VAT"] == -df_gas.loc[index, "tunover_VAT"]
                and df_gas.loc[id_, "purchase_cost_VAT"]
                == -df_gas.loc[index, "purchase_cost_VAT"]
            ):
                index_list.append(id_)
                break
            else:
                id_ -= 1

CPU times: user 129 ms, sys: 3.99 ms, total: 133 ms
Wall time: 130 ms


In [104]:
index_list

[402, 401, 586, 577, 597, 596]

### Вариант 2 антисортировка

In [107]:
%%time

# сотрируем dataframe по индексу в обратом порядке
# теперь запись об отмене покупки будет располагаться ближе к началу списка, чем запись о покупке
df_gas = df_gas.sort_index(ascending=False)
# сохраняем реквизиты покупки из первой строки в список
list_del = [
    df_gas.loc[df_gas.shape[0] - 1, "product"],
    df_gas.loc[df_gas.shape[0] - 1, "store"],
    df_gas.loc[df_gas.shape[0] - 1, "city"],
    df_gas.loc[df_gas.shape[0] - 1, "sold_quantity"],
]
# создаём пустой список для хранения индексов обречённых строк
index_list = []
# проходим по списку от максимального индекса к минимальному
for index, row in df_gas.iterrows():
    # если количество товара отрицательное (возврат):
    # а) заменяем элементы list_del на реквизиты этой "покупки":
    # б) сохраняем индекс строки в список обрачённых строк
    if row["sold_quantity"] < 0:
        list_del = [
            df_gas.loc[index, "product"],
            df_gas.loc[index, "store"],
            df_gas.loc[index, "city"],
            df_gas.loc[index, "sold_quantity"],
        ]
        index_list.append(index)
    # если реквизиты покупки совпадают с хранящимися в list_del, а количество товара противоположно:
    # а) сохраняем индекс строки в список обрачённых строк
    # б) очищаем list_del
    if (
        row["product"] == list_del[0]
        and row["store"] == list_del[1]
        and row["city"] == list_del[2]
        and row["sold_quantity"] == -list_del[3]
    ):
        index_list.append(index)
        list_del = [0, 0, 0, 0]

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 128 ms


In [108]:
index_list

[597, 596, 586, 577, 402, 401]